# ANSWER KEY: Debug Drill 06 - Wrong Threshold

**Bug:** Colleague uses the default 0.5 probability threshold, but the business can only call 200 customers per week.

**Key Lesson:** The optimal threshold depends on business constraints (capacity, cost), not a default 0.5.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_score, recall_score
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/shared/data/streamcart_customers.csv')

features = ['tenure_months', 'logins_last_30d', 'orders_last_30d', 'support_tickets_last_30d']
X = df[features].fillna(0)
y = df['churn_30d']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
model.fit(X_train, y_train)
probs = model.predict_proba(X_test)[:, 1]

## The Bug (Colleague's Code)

In [ ]:
# ===== BUGGY CODE =====
# Colleague uses default threshold
threshold = 0.5
predictions = (probs >= threshold).astype(int)

print(f"Using threshold = {threshold}")
print(f"Customers flagged for calls: {predictions.sum()}")
print(f"\nBusiness constraint: Can only call 200 customers/week")
print(f"\nProblem: We're flagging {predictions.sum()} customers but can only call 200!")

## Why This Is Wrong

**Business context matters:**
- The retention team has LIMITED CAPACITY (200 calls/week)
- Using threshold=0.5 ignores this constraint
- We might flag 500 customers but can only call 200

**What we actually need:**
- Select the TOP 200 highest-risk customers
- Find the threshold that gives us exactly 200 predictions
- Maximize precision within our capacity

In [ ]:
# Show the problem
print("Probability distribution:")
print(f"  Min: {probs.min():.3f}")
print(f"  Max: {probs.max():.3f}")
print(f"  Mean: {probs.mean():.3f}")

print(f"\nAt threshold=0.5: {(probs >= 0.5).sum()} customers flagged")
print(f"At threshold=0.3: {(probs >= 0.3).sum()} customers flagged")
print(f"At threshold=0.7: {(probs >= 0.7).sum()} customers flagged")

## The Fix

In [ ]:
# ===== FIXED CODE =====
# Use capacity-based selection (top K)

target_calls = 200  # Business constraint

# Method 1: Find threshold for top K
sorted_probs = np.sort(probs)[::-1]  # Sort descending
optimal_threshold = sorted_probs[target_calls - 1]  # 200th highest probability

print(f"Capacity-based threshold: {optimal_threshold:.3f}")
print(f"(This selects approximately the top {target_calls} customers)")

In [ ]:
# Method 2: Direct top-K selection (preferred)
top_k_indices = np.argsort(probs)[::-1][:target_calls]

# Create predictions
predictions_topk = np.zeros(len(probs))
predictions_topk[top_k_indices] = 1

print(f"\nTop-K Selection Results:")
print(f"  Customers selected: {int(predictions_topk.sum())}")
print(f"  Actual churners in selection: {y_test.iloc[top_k_indices].sum()}")
print(f"  Precision@{target_calls}: {y_test.iloc[top_k_indices].mean():.1%}")

In [ ]:
# Compare to baseline
base_rate = y_test.mean()
precision_at_k = y_test.iloc[top_k_indices].mean()
lift = precision_at_k / base_rate

print(f"\nComparison:")
print(f"  Base churn rate: {base_rate:.1%}")
print(f"  Precision@{target_calls}: {precision_at_k:.1%}")
print(f"  Lift: {lift:.1f}x better than random")

print(f"\nBusiness impact:")
print(f"  Random 200 calls: ~{int(200 * base_rate)} churners reached")
print(f"  Model's top 200: ~{int(200 * precision_at_k)} churners reached")

In [ ]:
# Self-check
assert int(predictions_topk.sum()) == target_calls, "Should select exactly K customers"
assert precision_at_k > base_rate, "Model should beat random"
print("\nPASS: Threshold correctly aligned to capacity constraint!")

## Threshold Selection Guide

| Constraint | Threshold Approach |
|------------|-------------------|
| Fixed capacity (K calls) | Select top K by probability |
| Cost-based (profit > 0) | Find threshold where expected value > 0 |
| Recall target (catch X%) | Find threshold that achieves X% recall |
| No constraint | Use 0.5 or calibrate based on precision/recall trade-off |

## Completed Postmortem

### What happened:
- Colleague used default 0.5 threshold, flagging more customers than the team can call
- Resulted in arbitrary selection of who to call, wasting model value

### Root cause:
- Threshold was chosen mathematically (0.5 = "more likely than not") rather than based on business constraint
- No discussion with business about capacity

### How to prevent:
- Always ask: "What's the constraint?" (capacity, budget, recall target)
- Use Precision@K when capacity is limited
- Report metrics at the actual operating point, not arbitrary thresholds